In [40]:
# !pip install paho-mqtt
# !pip install dnslib
# !pip install dnspython
# !python -m pip install --upgrade pip
!pip3 install pysocks==1.7.0

  Attempting uninstall: pysocks
    Found existing installation: PySocks 1.7.1
    Uninstalling PySocks-1.7.1:
      Successfully uninstalled PySocks-1.7.1


# DNSServer

In [1]:
from dns.resolver import Resolver
from http.server import SimpleHTTPRequestHandler
# from dnslib.zoneresolver import ZoneResolver
# from dnslib.dns import RR
from dnslib.server import DNSServer, DNSHandler, DNSLogger
import json
import paho.mqtt.client as mqtt
import time
import os

In [ ]:
broker_address = "broker.hivemq.com" # Free Available MQTT Broker
broker_port = 1883 # MQTT Default Port Number

client = mqtt.Client()
client.connect(broker_address, broker_port, 60) # 60s = Keep Alive interval is the duration to wait for ACK before retransmission. 

class MyHandler(SimpleHTTPRequestHandler):
        
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()

            if pre_state[f'btn1']:
                button = (f"""
                        <div class="form-check form-switchmx-auto">
                          <input id="btn1" onclick="buttonClicked('btn1')" class="form-check-input" checked type="checkbox" style="width: 9em; height: 3em;">
                        </div>
                        """)
            else:
                button = (f"""
                        <div class="form-check form-switchmx-auto">
                          <input id="btn1" onclick="buttonClicked('btn1')" class="form-check-input" type="checkbox" style="width: 9em; height:3em;">
                        </div>
                        """)

            html_content = """
            <!DOCTYPE html>
            <html lang="en">
            
            <head>
              <meta charset="UTF-8">
              <meta name="viewport" content="width=device-width, initial-scale=1.0">
              <title>Smart Plug</title>
              <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css" rel="stylesheet"
              integrity="sha384-4bw+/aepP/YC94hEpVNVgiZdgIC5+VKNBQNGCHeKRQN+PtmoHDEXuppvnDJzQIu9" crossorigin="anonymous">
            </head>
            <body>

            <div class="container text-center">
              <br/>
              <h2><strong>Smart Plug</strong></h2>
              <div class="text-center">
                <br/>
                <h5>Toggle the button to control the switch:</h5>
                <br/><br/>
                <div>
                  """ + button + """
                </div>
              </div>
            </div>

            <script>
              function buttonClicked(buttonId) {
                var checkbox = document.getElementById(buttonId);
                var xhr = new XMLHttpRequest();
                xhr.open("GET", "/" + buttonId + "," + checkbox.checked,true);
                xhr.send();
              }
            </script>

            <script
              src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/js/bootstrap.bundle.min.js"
              integrity="sha384-HwwvtgBNo3bZJJLYd8oVXjrBZt8cqVSpeBNS5n7C8IVInixGAoxmnlMuBnhbgrkm" crossorigin="anonymous">
            </script>

            </body>
            </html>
            """
            self.wfile.write(html_content.encode())
            
        else:
            button_id = self.path[1:] # Remove the leading '/'
            self.handle_button_click(button_id)

    def handle_button_click(self, btn_id):
        if btn_id[0:3] == 'btn':
            btn, state = btn_id.split(',')
            s = 0
            pre_state[btn] = False
            if state == 'true':
                pre_state[btn] = True
                s = 1
            with open('states.json', 'w') as file:
                file.write(json.dumps(pre_state))

            topic = "CN/CEP"
            message = f"{btn}{s}"
            
            client.publish(topic, message)
            print(f"Button {btn} state: {state}")
            
class TestResolver:
    def resolve(self, request, handler):
        reply = request.reply()
        reply.add_answer(*RR.fromZone("cep_smartplug.cn. 8222 A 192.168.10.13"))
        return reply
    
if __name__ == "__main__":
    IP = "192.168.10.13"
    PORT = 8111
    DOMAIN = "cep_smartplug.cn"
    
    if not os.path.isfile('states.json'): # Make json file to save pre_state of switch, if not exists
        pre_state = {"btn1": False} # Switch off by default
        with open('states.json', 'w') as file:
            file.write(json.dumps(pre_state))
    else: # Loads pre_state from existing json file
        with open('states.json') as file: 
            pre_state = json.loads(file.read())

#     resolver = Resolver()
#     resolver.nameservers = ['8.8.8.8']
#     resolver.domain = '192.168.10.13'
#     resolver.resolve_name = DOMAIN

#     zone = '''
# {dnsdomain}       IN      A       {serverip}
# '''.format(dnsdomain=DOMAIN, serverip=IP)
#     resolver = ZoneResolver(zone, True)

    resolver = TestResolver()

    logger = DNSLogger(prefix=False)
    server = DNSServer(resolver,logger=logger,tcp=True,
                       address=IP,port=PORT,handler=MyHandler)

    server.start_thread()
    server.server.serve_forever()
    print(server.isAlive())
#     server.stop()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19420\544961510.py:4: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
